In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['KMP_WARNINGS'] = 'off'
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import re

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

In [7]:
np.stack([ np.random.permutation(8) for _ in range(100000) ]).shape

(100000, 8)

In [2]:
signal = ObjIter([Tree(f, altfile='test_{base}') for f in fc.eightb.feynnet.feynnet_signal_list[25:27]])

100%|██████████| 1/1 [00:00<00:00,  1.77it/s]


In [3]:
signal.apply(lambda tree : eightb.load_feynnet_assignment(tree, model=eightb.models.feynnet_bkg_33sig.storage))

[None, None]

In [4]:
signal = signal.apply(EventFilter('eighbt',filter=lambda t:t.nfound_select==8))

In [5]:
signal.apply(lambda tree : eightb.load_true_assignment(tree))

[None, None]

In [6]:
def get_h_j_dr(tree, prefix=''):
    if prefix and not prefix.endswith('_'): prefix += '_'
    j_p4 = build_p4(tree, prefix=prefix+'j')

    j1_p4, j2_p4 = j_p4[:,::2], j_p4[:,1::2]
    h_j_dr = calc_dr_p4(j1_p4, j2_p4)

    tree.extend(
        **{f'{prefix}h_j_dr': h_j_dr}
    )

In [7]:
signal.apply(get_h_j_dr)
signal.apply(lambda t : get_h_j_dr(t, prefix='true'))

[None, None]

In [8]:
def get_y_h_dr(tree, prefix=''):
    if prefix and not prefix.endswith('_'): prefix += '_'
    h_p4 = build_p4(tree, prefix=prefix+'h')

    h1_p4, h2_p4 = h_p4[:,::2], h_p4[:,1::2]
    y_h_dr = calc_dr_p4(h1_p4, h2_p4)

    tree.extend(
        **{f'{prefix}y_h_dr': y_h_dr}
    )

In [9]:
signal.apply(get_y_h_dr)
signal.apply(lambda t : get_y_h_dr(t, prefix='true'))

[None, None]

In [10]:
signal.apply(lambda t : ak.mean(t.h_signalId>-1,axis=0))

[<Array [0.805, 0.709, 0.792, 0.711] type='4 * ?float64'>, <Array [0.983, 0.983, 0.983, 0.984] type='4 * ?float64'>]

In [11]:
signal.apply(lambda t : ak.mean(t.y_signalId>-1,axis=0))

[<Array [0.451, 0.452] type='2 * ?float64'>, <Array [0.981, 0.981] type='2 * ?float64'>]

In [27]:
def write_mass(t):
    h_eff = ak.mean(t.h_signalId>-1,axis=0)
    y_eff = ak.mean(t.y_signalId>-1,axis=0)

    newtree = dict(
        mx=t.mx,
        my=t.my,
        **{
            f'h_eff'  : h_eff
        },
        **{
            f'y_eff'  : y_eff
        },
    )

    with ut.recreate(f'MX_{t.mx}_MY_{t.my}.root') as f:
        f['tree'] = {
            k : [v] for k,v in newtree.items()
        }
    # return newtree

In [28]:
signal.apply(write_mass)

[None, None]